In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
data_url = "https://raw.githubusercontent.com/fenago/tf/main/Chapter5-Classification_Models/datasets/accelerometer.csv"

data = pd.read_csv(data_url, sep=",")
data.head()

,wconfid,pctid,x,y,z
0,1,20,1.004,0.090,-0.125
1,1,20,1.004,-0.043,-0.125
2,1,20,0.969,0.090,-0.121
3,1,20,0.973,-0.012,-0.137
4,1,20,1.000,-0.016,-0.121


In [3]:
data['wconfid'].value_counts()

1    51000
2    51000
3    51000
Name: wconfid, dtype: int64

In [4]:
reduce_map = {"wconfid": {1:0, 2:1, 3:2} }
data.replace(reduce_map, inplace=True)

In [5]:
data['wconfid'].value_counts()

0    51000
1    51000
2    51000
Name: wconfid, dtype: int64

In [6]:
data.dropna(inplace=True)

In [7]:
y = data.pop('wconfid')
X = data

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, 
                                                  stratify=y,random_state=42)

In [9]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(107100, 4)
(45900, 4)
(107100,)
(45900,)


In [10]:
y_train.head()

74339     1
144853    2
48273     0
75741     1
122914    2
Name: wconfid, dtype: int64

In [11]:
import tensorflow as tf

In [12]:
tf.random.set_seed(8)

In [13]:
from tensorflow.keras.layers import Dense

In [14]:
def model_builder(hp):
        model = tf.keras.Sequential()
        hp_l2 = hp.Choice('l2', values = [0.1, 0.01, 0.001])
        hp_units = hp.Int('units', min_value=128, max_value=512, step=64)
        reg_fc1 = Dense(hp_units, input_shape=(4,), activation='relu', \
                        kernel_regularizer=tf.keras.regularizers\
                                             .l2(l=hp_l2))
        reg_fc2 = Dense(512, activation='relu', \
                        kernel_regularizer=tf.keras.regularizers\
                                             .l2(l=hp_l2))
        reg_fc3 = Dense(128, activation='relu', \
                        kernel_regularizer=tf.keras.regularizers\
                                             .l2(l=hp_l2))
        reg_fc4 = Dense(128, activation='relu', \
                        kernel_regularizer=tf.keras.regularizers\
                                             .l2(l=hp_l2))
        reg_fc5 = Dense(3)
        model.add(reg_fc1)
        model.add(reg_fc2)
        model.add(reg_fc3)
        model.add(reg_fc4)
        model.add(reg_fc5)
        loss = tf.keras.losses.SparseCategoricalCrossentropy()
        hp_learning_rate = hp.Choice('learning_rate', \
                                     values = [0.01, 0.001])
        optimizer = tf.keras.optimizers.Adam(hp_learning_rate)
        model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
        return model

In [15]:
import keras_tuner as kt

In [18]:
tuner = kt.BayesianOptimization(model_builder, \
                                    objective = 'val_accuracy', \
                                    max_trials = 10, overwrite=True)

In [19]:
tuner.search(X_train, y_train, \
                 validation_data=(X_test, y_test))

Trial 10 Complete [00h 00m 09s]
val_accuracy: 0.3333333432674408

Best val_accuracy So Far: 0.3333333432674408
Total elapsed time: 00h 01m 32s
INFO:tensorflow:Oracle triggered exit


In [20]:
best_hps = tuner.get_best_hyperparameters()[0]

In [21]:
best_units = best_hps.get('units')
best_units

384

In [22]:
best_lr = best_hps.get('learning_rate')
best_lr

0.01

In [23]:
best_l2 = best_hps.get('l2')
best_l2

0.001

In [25]:
model = tuner.hypermodel.build(best_hps)
model.fit(X_train, y_train, epochs=5, \
              validation_data=(X_test, y_test))

Epoch 1/5
3347/3347 [==============================] - 9s 3ms/step - loss: 10.8393 - accuracy: 0.3334 - val_loss: 10.9096 - val_accuracy: 0.3333
Epoch 2/5
3347/3347 [==============================] - 8s 3ms/step - loss: 1.5029 - accuracy: 0.3331 - val_loss: 1.0986 - val_accuracy: 0.3333
Epoch 3/5
3347/3347 [==============================] - 8s 3ms/step - loss: 1.0986 - accuracy: 0.3333 - val_loss: 1.0986 - val_accuracy: 0.3333
Epoch 4/5
3347/3347 [==============================] - 9s 3ms/step - loss: 1.0986 - accuracy: 0.3333 - val_loss: 1.0986 - val_accuracy: 0.3333
Epoch 5/5
3347/3347 [==============================] - 8s 2ms/step - loss: 1.0986 - accuracy: 0.3333 - val_loss: 1.0986 - val_accuracy: 0.3333
